In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
df=pd.read_csv("../input/30days-10folds/train_10_folds.csv")
df_test=pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission=pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

In [ ]:
df

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
num_cols= [col for col in useful_features if 'cont' in col]
df_test = df_test[useful_features]

In [ ]:
df_test


In [ ]:
final_predictions = []
scores = []
for fold in range(10):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ohe = preprocessing.OneHotEncoder(sparse=False, handle_unknown="ignore")
    xtrain_ohe = ohe.fit_transform(xtrain[object_cols])
    xvalid_ohe = ohe.transform(xvalid[object_cols])
    xtest_ohe = ohe.transform(xtest[object_cols])
    
    xtrain_ohe = pd.DataFrame(xtrain_ohe, columns=[f"ohe_{i}" for i in range(xtrain_ohe.shape[1])])
    xvalid_ohe = pd.DataFrame(xvalid_ohe, columns=[f"ohe_{i}" for i in range(xvalid_ohe.shape[1])])
    xtest_ohe = pd.DataFrame(xtest_ohe, columns=[f"ohe_{i}" for i in range(xtest_ohe.shape[1])])
    
    xtrain = pd.concat([xtrain, xtrain_ohe], axis=1)
    xvalid = pd.concat([xvalid, xvalid_ohe], axis=1)
    xtest = pd.concat([xtest, xtest_ohe], axis=1)
    
    
    xtrain = xtrain.drop(object_cols, axis=1)
    xvalid = xvalid.drop(object_cols, axis=1)
    xtest = xtest.drop(object_cols, axis=1)
    
    Scaler = preprocessing.StandardScaler()
    xtrain[num_cols] =  Scaler.fit_transform(xtrain[num_cols])
    xvalid[num_cols] =  Scaler.transform(xvalid[num_cols])
    xtest[num_cols] =  Scaler.transform(xtest[num_cols])
    
    
    model = XGBRegressor(random_state=fold, tree_method='gpu_hist',n_estimators=1000, learning_rate=0.05, gpu_id=0, predictor="gpu_predictor")
    model.fit(xtrain, ytrain,early_stopping_rounds=4,eval_set=[(xvalid,yvalid)],verbose=False)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
preds

In [ ]:
sample_submission.target = preds
sample_submission.to_csv("submission.csv", index=False)